# ph21: HWK 1
In this assignment, we will get Buoy data from the NDBC and then use it to make some estimates about the future.

### Familiarize yourself

We will be using data from the National Data Buoy Center (NDBC). Visit the NDBC website, https://www.ndbc.noaa.gov, and access some historical data. Take a look at the URL for the data you have accessed: you will need this format to access the data from python.

### Part 1: Getting the data

You will need to save all necessary for your second part here. The following steps are for reference.

1. Use the `requests` library (or any other similar library at your choice) to download historical Buoy data
3. Save all data you downloaded into (a) file(s). If you feel like it, only save that part that is relevant for your second part.
4. Use a html parser library (e.g., `BeautifulSoup`, `lxml`, or `html5lib`) to extract the meaning of the data (i.e. what is it measuring and what are the units?). Also save this information somewhere.
5. From now on, you don't need to make any more requests to the NDBC website. Only use the data you downloaded.


### Part 2: Processing the data
6. (Optional) use the `pandas` library to read the data and process as needed, and save the processed data if you want to. Save the processed data
7. plot the oceant temperatures (`WTMP`), wave heights (`WVHT`), average wave periods (`APD`), and wind speeds (`WSPD`) going back 10 years or so (some buoys don't have all the data every year)
8. Look through `scipy.stats` and choose something like Pearson's or Spearman's correlation test. Determine what (if any) correlations you find between mean ocean temperatures and maximum wave heights or wave periods.


### All written codes are only here for reference. You can also modify them if you want to.

# Part 1

In [1]:
# Libraries you might want to use
# Uncomment the ones you need
from datetime import datetime
from io import StringIO
from pathlib import Path
import csv 
import requests
from requests import Session
# from curl_cffi.requests import Session, AsyncSession

from bs4 import BeautifulSoup
import matplotlib.dates as mdates
# from lxml import html

from matplotlib import pyplot as plt
# import seaborn as sns

import pandas as pd
# import numpy as np
from scipy.stats import pearsonr

import yaml
#from tqdm import tqdm
import glob

In [3]:
# Prepare requests session. Let's pretend we are Chrome 118 here

# Think: In some cases, we can still be blocked as been flagged as bot at the first request.
#        Can you guess how the server knows it? (ignore javascript for now)

sess = Session()
sess.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'})

# Think: What's the difference between using requests.get() and sess.get()?

In [5]:
# Where to save the fetched data
path = Path('./Data')
path.mkdir(exist_ok=True)
url_template = 'https://www.ndbc.noaa.gov/view_text_file.php?filename={buoy_id}h{year}.txt.gz&dir=data/historical/stdmet/'
description_page_url = 'https://www.ndbc.noaa.gov/faq/measdes.shtml'

In [7]:
# Load the config file
with open('config.yaml') as f:
    config = yaml.safe_load(f)

buoy_ids:list[int] = config['buoys'] # We only use these id's for this assignment
head_columns:list[str] = config['columns'] # Data columns we want to keep. You may add more if you want

In [9]:
# An example of how to fetch the description and put it in a pd.DataFrame

r = sess.get(url_template.format(buoy_id=41010, year=2022))
r.raise_for_status()
buf= StringIO(r.text)
df = pd.read_csv(buf, sep=r'\s+', skiprows=[1])

Take a look:

In [12]:
df.head()

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,2022,1,1,0,0,222,1.1,1.5,99.00,99.00,99.0,999,1019.6,24.9,25.2,20.6,99.0,99.0
1,2022,1,1,0,10,212,1.5,1.9,99.00,99.00,99.0,999,1019.7,24.9,25.2,20.5,99.0,99.0
2,2022,1,1,0,20,208,1.4,1.9,99.00,99.00,99.0,999,1019.6,24.9,25.2,20.4,99.0,99.0
3,2022,1,1,0,30,204,1.4,1.9,99.00,99.00,99.0,999,1019.7,24.9,25.2,20.5,99.0,99.0
4,2022,1,1,0,40,205,1.3,1.9,0.62,13.79,4.6,28,1019.8,24.9,25.1,20.6,99.0,99.0


Now, your task here is:
1. For each buoy_id, generate all urls pointing to its data, and fetch&save the data. If the data is missing (`status_code`=404), skip it.
2. Save everything in raw, or any processed form that you see fits.
3. If the file is already downloaded, don't download it again. Just skip it here.

Learn more about http status codes here:
- https://en.wikipedia.org/wiki/List_of_HTTP_status_codes
- https://http.cat/

Two important ones here for today:
- 200: OK; everything's alright
- 404: Not Found; the file is missing

In [15]:
# ================ INSERT YOUR CODE HERE ================
for b_id in buoy_ids:
    for b_year in range(2014,2025):
        try:
            r = sess.get(url_template.format(buoy_id=b_id, year=b_year))
            r.raise_for_status()
            buf= StringIO(r.text)
            df = pd.read_csv(buf,sep=r'\s+', skiprows=[1])
            folder = Path(f'./Data/buoy_{b_id}')
            folder.mkdir(exist_ok=True)
            if not (path/f'./Data/buoy_{b_id}/buoy_{b_id}_{b_year}.csv').exists():
                df.to_csv(f'./Data/buoy_{b_id}/buoy_{b_id}_{b_year}.csv',index=False,columns=head_columns)
        except requests.exceptions.HTTPError as e:
            if r.status_code == 404:
                with open('./Data/missing_data.txt','a') as file:
                    file.write(f'{b_id} : {b_year}\n')
            continue


Though the units for all quantities are included in the data files received already, lets also try get them again from the website along with the description for each columns by parsing htmls.

All information of our interest is available at [`description_page`](https://www.ndbc.noaa.gov/faq/measdes.shtml#stdmet).

Your task is to:

1. Get a dictionary of all the quantity name and its unit
2. Get a dictionary of all the quantity name and its description

To achieve this, you may want to use some html parser library, such as `BeautifulSoup`, `lxml`, or `html5lib`. If you really feel like it, you can also use regex just for fun like the good old days.

Tip: Having no idea where to start? Try to open the [`description_page`](https://www.ndbc.noaa.gov/faq/measdes.shtml#stdmet) in your browser, and use the `inspect` tool (F12, or Ctrl+Shift+I) to see how the html is structured and locate the part of your interest. Pasting a copy of the html to some LLM and ask it how to extract the information you want may also be a good idea.

Another Tip: Something called [`xpath`](https://en.wikipedia.org/wiki/XPath) may be extremely useful here. It can be access with `lxml`.

In [18]:
r = sess.get(description_page_url)
r.raise_for_status()
html_text = r.text # raw html text for your input

In [20]:
# ================ INSERT YOUR CODE HERE ================
dsc = BeautifulSoup(html_text,'html')
units = {}
meaning = {}

# parsing all glossaries except for the last one (exluding abbreviation list) 

for dl in dsc.find_all('dl')[0:-1]:
    for element in dl.find_all(['dt', 'dd']):
        if element.name == 'dt':
            meaning[element.get_text(strip=True)]=''
            last_key = element.get_text(strip=True)
        elif element.name == 'dd':
            meaning[last_key] = element.get_text()
            
for pre in dsc.find_all('pre'):
    units_loc = pre.get_text().find('#',2)
    if units_loc == -1:
        continue
    names = pre.get_text()[2:units_loc]
    names = names.split()
    unts_str = pre.get_text()[units_loc+1:]
    unts_lst = unts_str.split(maxsplit=len(names))[:-1]
    for i in range(len(names)):
        if names[i] not in units.keys():
            units[names[i]]=unts_lst[i]

if not (path/'units.yaml').exists() or not (path/'meanings.yaml').exists():
    ...
    with open(path/'units.yaml', 'w') as f:
        yaml.dump(units, f)
    with open(path/'meanings.yaml', 'w') as f:
        yaml.dump(meaning, f)

In [22]:
with open(path/'units.yaml') as f:
    unit_map = yaml.safe_load(f)
unit_map

{'%Good3': '%',
 '%Good4': '%',
 '%GoodE': '%',
 '...>': '...>',
 'ACCUM': 'mm',
 'APD': 'sec',
 'ATMP': 'degC',
 'BATTCURR': 'Amps',
 'BATTTEMP': 'DegC',
 'BATTV': 'Volts',
 'Bin': '-',
 'CHILL': 'degC',
 'CLCON': 'ug/l',
 'CM1': '-',
 'CM2': '-',
 'CM3': '-',
 'CM4': '-',
 'COND': 'mS/cm',
 'DD': 'dy',
 'DEP01': 'm',
 'DEP02': 'm',
 'DEP03': 'm',
 'DEPTH': 'm',
 'DEWP': 'degC',
 'DIR01': 'degT',
 'DIR02': 'degT',
 'DIR03': 'degT',
 'DPD': 'sec',
 'Depth': 'm',
 'Dir': 'degT',
 'EH': 'mv',
 'EI1': '-',
 'EI2': '-',
 'EI3': '-',
 'EI4': '-',
 'ErrVl': 'cm/s',
 'Flags': '-',
 'GDR': 'degT',
 'GST': 'm/s',
 'GTIME': 'hhmm',
 'HEAT': 'degC',
 'HEIGHT': 'm',
 'I': '-',
 'ICE': 'cm/hr',
 'LAT': 'deg',
 'LON': 'deg',
 'LWRAD': 'w/m2',
 'MM': 'mo',
 'MWD': 'degT',
 'O2%': '%',
 'O2PPM': 'ppm',
 'OTMP': 'degC',
 'PCT': '%',
 'PH': '-',
 'PRES': 'hPa',
 'PTDY': 'hPa',
 'PTIME': 'hhmm',
 'RATE': 'mm/h',
 'REMCAP': 'Ah',
 'SAL': 'psu',
 'SDEV': '-',
 'SPD01': 'cm/s',
 'SPD02': 'cm/s',
 'SPD03': '

In [24]:
with open(path/'meanings.yaml') as f:
    meaning_map = yaml.safe_load(f)
meaning_map

{'': ' Notes:\nThe R1 and R2 values in the monthly and yearly historical data files are scaled by 100, a carryover from how the data are transported to the archive centers. The units are hundredths, so the R1 and R2 values in those files should be multiplied by 0.01.\nD(f,A) can take on negative values because of the trigonometric sine and cosine functions. There are several approaches to prevent or deal with the negative values. For more information and discussion of some approaches see:  Use of advanced directional wave spectra analysis methods, M. D. Earle,  K. E. Steele, and D. W. C. Wang, Ocean Engineering, Volume 26, Issue 12, December 1999, Pages 1421-1434.\nALPHA2 has ambiguous results in using the arctangent function with the Fourier Coefficients,b 2 ,a 2 . When necessary, NDBC adds 180 degrees to ALPHA2 in order to minimize the difference between ALPHA 1 and ALPHA2.\n',
 '%Good3': 'The percentage of three-beam solutions that are good.',
 '%Good4': 'The percentage of four-beam

You may want to concatenate each buoy's data into a single dataframe, or any other data structure that you see fit. `pandas` is recommended here for your convenience.

After plotting, you will notice that some values are saturated at ~100. This is considered as artifacts, and should be removed. We will filter out everything $\ge 90$ in this case. If you are using `pandas`, you can do something like `df[df[columns] < 90]`.
Also, remove all rows with `NaN` values.

In [32]:
# ================ INSERT YOUR CODE HERE ================

# Concatenate the data of each Buoy 

for fldr in [p for p in Path('./Data').iterdir() if p.is_dir()]:
    if Path(f'./{fldr}/merged_data.csv').exists():
        continue
    csv_files = glob.glob(f'./{fldr}/*.csv')
    if csv_files: 
        data_frame = pd.concat(map(pd.read_csv, csv_files), ignore_index=True)
        for column in head_columns[5:]:
            data_frame = data_frame[data_frame[column] < 90 ]
            data_frame = data_frame[data_frame[column] != 'NaN']
        data_frame.to_csv(f'./{fldr}/merged_data.csv',index=False)

    


As python handles datetime objects in fundamentally nasty way, we provide a snippet to convert YMDhm to datetime objects for you. You can use it like this:

```python
YMDhm = df[['#YY','MM','DD','hh','mm']]
timestemp = YMDhm.agg(lambda x: datetime(*x), axis=1)
```

to get a series of datetime objects from the dataframe `df` with columns `#YY`, `MM`, `DD`, `hh`, and `mm`.

#### For one buoy, plot all of its columns from all last 10 years. You can use any plotter of your choice.
##### Can you see any trend? If it is too noisy for human eyes, what could to be done to make it more clear?

##### Launch `scipy.stats` and choose something like Pearson's or Spearman's correlation test. Determine what (if any) correlations you find between mean ocean temperatures and maximum wave heights or wave periods.|

In [41]:
# ================ INSERT YOUR CODE HERE ================
        
# DATA VISUALIZATION

# Getting the timestamps for each file
for fldr in [p for p in Path('./Data').iterdir() if p.is_dir()]:
    csv_file = open(f'./{fldr}/merged_data.csv','r')
    reader = csv.reader(csv_file)
    header = next(reader)
    timestamps = []
    for row in reader:
        time_str = ''
        for time in row[:5]:
            time_str += time + '-'
        timestamp = datetime.strptime(time_str, '%Y-%m-%d-%H-%M-') 
        timestamps.append(timestamp)
    csv_file.close()

    # Plotting each column and saving it in a jpg file
    for column in head_columns[5:]:
        if Path(f'./{fldr}/{str(fldr)[-5:]}_{column}.jpg').exists():
            continue
        df = pd.read_csv(f'./{fldr}/merged_data.csv')
        saved_column = df[column].tolist()
        plt.figure(figsize=(21,14))  
        plt.scatter(timestamps,saved_column,color='blue',edgecolor='none',s=2)
        plt.title(f'Buoy ID: {str(fldr)[-5:]}',fontsize=14)
        plt.xticks(rotation=45,fontsize=9)
        plt.xlabel('Time (year)',fontsize=14)
        plt.ylabel(f'{column} ({units[column]})',fontsize=18)
        plt.tick_params(axis='both',which='major',labelsize=18)
        plt.savefig(f'./{fldr}/{str(fldr)[-5:]}_{column}.jpg',bbox_inches='tight')
        plt.clf()
        plt.close()

    # Pearson's test
    df = pd.read_csv(f'./{fldr}/merged_data.csv')
    temp_column = df['WTMP'].tolist()
    for column in head_columns[6:8]:
        test_column = df[column].tolist()
        corr, p_value = pearsonr(temp_column, test_column)
        with open(f'./{fldr}/pearson_results.txt','a') as test:
            test.write(f'Pearson\'s test result for WTMP-{column}:\n' + 'Correlation Coefficient: '+str(corr)
                       +'\n'+'Statistical Significance: ' +str(p_value)+'\n')
        

            
    
